<a href="https://colab.research.google.com/github/Thandeka20/High-spatial-resolution-imagery-vs-high-spectral-imagery/blob/main/Code/Sentinel_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import google earth engine API**

In [2]:
import ee

**Authenticate google earth engine and your intialize the cloud project**

In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='thandeka-skosana')

**Import libraries needed to run the code**

In [4]:
#import modules
!pip install rasterio
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import geemap
import re
import os
import gdown
from IPython.display import Image, display
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 40.0 MB/s eta 0:00:00
Mounted at /content/drive


**Visualize Sentinel-2 Scene**

In [5]:
#visualize the Sentinel-2 scene
# Define the bands to use for visualization
bands = ee.List(["B4", "B3", "B2"])

# Define visualization parameters
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
}

# Example: Display an image using the defined visualization parameters
image = ee.Image('COPERNICUS/S2/20230727T073621_20230727T075808_T36JTT')
display_image = Image(url=image.visualize(**vis_params).getThumbURL({'dimensions': '500x500'}))
display(display_image)

**Compute vegetation indices**

In [6]:
# Define vegetation indices
GI = image.expression('(GREEN)/(RED)', {
    'GREEN': image.select('B3'),
    'RED': image.select('B4'),
}).rename('GI')

IRG = image.expression('(RED-GREEN)', {
    'RED': image.select('B4'),
    'GREEN': image.select('B3'),
}).rename('IRG')

NGRDI = image.expression('(GREEN-RED)/(GREEN+RED)', {
    'RED': image.select('B4'),
    'GREEN': image.select('B3'),
}).rename('NGRDI')

VARI = image.expression('(GREEN-RED)/(GREEN+RED+BLUE)', {
    'RED': image.select('B3'),
    'GREEN': image.select('B4'),
    'BLUE': image.select('B2'),
}).rename('VARI')

VDVI = image.expression('(2*GREEN-RED-BLUE)/(2*GREEN+RED+BLUE)', {
    'RED': image.select('B4'),
    'GREEN': image.select('B3'),
    'BLUE': image.select('B2'),
}).rename('VDVI')

WBI = image.expression('(Blue - Red) / (Blue + Red)', {
    'Blue': image.select('B2'),
    'Red': image.select('B4'),
}).rename('WBI')

# Add computed variables as bands to the image_final
image_final = image.addBands([GI, IRG, NGRDI, VARI, VDVI, WBI])
# Print image bands
print(image_final.getInfo())

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [1830, 1830], 'crs': 'EPSG:32736', 'crs_transform': [60, 0, 199980, 0, -60, 7300000]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32736', 'crs_transform': [10, 0, 199980, 0, -10, 7300000]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32736', 'crs_transform': [10, 0, 199980, 0, -10, 7300000]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32736', 'crs_transform': [10, 0, 199980, 0, -10, 7300000]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [5490, 5490], 'crs': 'EPSG:32736', 'crs_transform': [20, 0, 199980, 0, -20, 7300000

**Load training data from google drive to google colab**

In [20]:
# Specify the Google Drive file ID of the shapefile
file_id = '1iA9L6tn0dwVlABPngFAgjRxd8xXET1c4'
output_path = '/content/drive/My Drive/Colab Notebooks/Data/TrainingSet_SabieCroc/Training_setSc.shp'
url = f'https://drive.google.com/uc?id={file_id}'

# Read the shapefile into a GeoDataFrame
training_data = gpd.read_file(output_path)

print (training_data)
#cover shp file to geojson
#training_set=training_data.to_file('training_set.geojson', driver='GeoJSON')

# Display the GeoDataFrame
#print(training_set)

# Create an Earth Engine FeatureCollection from the list of dictionaries
#training_data_dict = training_data.to_dict('records')

# Print the type of training_data
#print(type(training_data_dict))

#convert geodataframe to a feature collection
#training_subset = ee.FeatureCollection(training_set)

                                       Name  \
0     0e9fd6a4c5704984a97731f76c9fdc86.jpeg   
1     104fe7223f3543ddbbb304faaac44eaf.jpeg   
2     12c5a6fe83ed45d388722f623b9dfc7c.jpeg   
3     158cb3a73f824c7da590cb5dacbfaa7c.jpeg   
4     4105e29b7b9743539045ff4992cb673d.jpeg   
...                                     ...   
3124                                   4458   
3125                                   4459   
3126                                   4463   
3127                                   4464   
3128                                   4465   

                                                   Path  GPS_ID    ID  \
0     C:\Users\Liam\Desktop\Sabie_Croc\Day_10\0e9fd6...    3773   2.0   
1     C:\Users\Liam\Desktop\Sabie_Croc\Day_10\104fe7...    3656   2.0   
2     C:\Users\Liam\Desktop\Sabie_Croc\Day_10\12c5a6...    3670   2.0   
3     C:\Users\Liam\Desktop\Sabie_Croc\Day_10\158cb3...    3737   2.0   
4     C:\Users\Liam\Desktop\Sabie_Croc\Day_10\4105e2...    3726   2.0 

**Sample regions from the image at training data locations**

In [8]:
#select bands
bands= ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8','B8A', 'B9', 'B11', 'B12','GI', 'IRG', 'NGRDI', 'VARI', 'VDVI', 'WBI',];

# Sample regions from the image at training feature locations
training_data = image_final.select(bands).sampleRegions(
    collection=training_data,
    properties=['ID', 'LULC_Class'],
    scale=10
)

EEException: Unrecognized argument type to convert to a FeatureCollection:                                        Name  \
0     0e9fd6a4c5704984a97731f76c9fdc86.jpeg   
1     104fe7223f3543ddbbb304faaac44eaf.jpeg   
2     12c5a6fe83ed45d388722f623b9dfc7c.jpeg   
3     158cb3a73f824c7da590cb5dacbfaa7c.jpeg   
4     4105e29b7b9743539045ff4992cb673d.jpeg   
...                                     ...   
3124                                   4458   
3125                                   4459   
3126                                   4463   
3127                                   4464   
3128                                   4465   

                                                   Path  GPS_ID    ID  \
0     C:\Users\Liam\Desktop\Sabie_Croc\Day_10\0e9fd6...    3773   2.0   
1     C:\Users\Liam\Desktop\Sabie_Croc\Day_10\104fe7...    3656   2.0   
2     C:\Users\Liam\Desktop\Sabie_Croc\Day_10\12c5a6...    3670   2.0   
3     C:\Users\Liam\Desktop\Sabie_Croc\Day_10\158cb3...    3737   2.

**Run the classification using random forest**

In [ ]:
#Classification
classifier = ee.Classifier.smileRandomForest(100).train(
    features=training_data,
    classProperty='ID',
    inputProperties=bands
)
classified = image_final.select(bands).classify(classifier)

**Defining a colour palette for the classified map and viweing the map**

In [ ]:
# Define the palette
SabieCrocPalette = [
    '351C75', 'F91DF9', '980A7D', '741b47', 'fd0618', 'E06666', 'ffcc99', 'ffffff', '999999', 'a8a800',
    '6aa84f', '14870e', 'DB992D', 'ff7f00', '000000', '0a14f9', '08f3e4'
]

# Visualization parameters
viz = {
    'min': 1,
    'max': 17,
    'palette': SabieCrocPalette
}

# Create a Map object using geemap
m = geemap.Map(center=[20, 0], zoom=3)

# Add the classified image to the map
m.addLayer(classified, viz, 'Classification')
m.centerObject(classified, zoom=10)

# Display the legend
m.add_legend(title='Classification', colors=SabieCrocPalette, labels=[str(i) for i in range(1, 18)])

# Display the map
m


